In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import csv

iris = []
with open("iris.csv", "r") as f:
    for row in csv.DictReader(f):
        iris.append(row)

In [ ]:
y_col = "class"

# feature_names = ["sepal_len", "petal_len", "sepal_width", "petal_width"]
feature_names = []
for col in iris[0].keys():
    if col not in y_col:
        feature_names.append(col)

class_names = []
for row in iris:
    if row[y_col] not in class_names:
        class_names.append(row[y_col])

x = []
for row in iris:
    r = []
    for feature in feature_names:
        r.append(float(row[feature]))
    x.append(r)
x = np.array(x)

y = []
for row in iris:
    name = row[y_col]
    if name == "Iris-virginica":
        y.append(0)
    elif name == "Iris-versicolor":
        y.append(1)
    elif name == "Iris-setosa":
        y.append(2)

y = np.array(y)

In [ ]:
enc = OneHotEncoder()
y_encoded = enc.fit_transform(y[:, np.newaxis]).toarray()

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Split the data set into training and testing
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, y_encoded, test_size=0.3, random_state=2)

n_features = x.shape[1]
n_classes = y_encoded.shape[1]

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
for target, target_name in enumerate(class_names):
    x_plot = x[y == target]
    plt.plot(x_plot[:, 0], x_plot[:, 1], linestyle='none', marker='o', label=target_name)
plt.xlabel(feature_names[0])
plt.ylabel(feature_names[1])
plt.axis('equal');

plt.subplot(1, 2, 2)
for target, target_name in enumerate(class_names):
    print(target)
    x_plot = x[y == target]
    plt.plot(x_plot[:, 2], x_plot[:, 3], linestyle='none', marker='o', label=target_name)
plt.xlabel(feature_names[2])
plt.ylabel(feature_names[3])
plt.axis('equal');

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model1 = Sequential(name="model_1")
for i in range(1):
    model1.add(Dense(8, input_dim=n_features, activation='relu'))
model1.add(Dense(n_classes, activation='softmax'))

model1.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model2 = Sequential(name="model_2")
for i in range(2):
    model2.add(Dense(8, input_dim=n_features, activation='relu'))
model2.add(Dense(n_classes, activation='softmax'))

model2.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model3 = Sequential(name="model_3")
for i in range(3):
    model3.add(Dense(8, input_dim=n_features, activation='relu'))
model3.add(Dense(n_classes, activation='softmax'))

model3.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

models = [model1, model2, model3]

for i in range(len(models)):
    models[i].summary()

In [ ]:
history_dict = {}

for model in models:
    print('Model name:', model.name)
    history = model.fit(x_train, y_train,
                                 batch_size=5,
                                 epochs=50,
                                 verbose=0,
                                 validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    history_dict[model.name] = [history, model]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 6))

for model_name in history_dict:
    val_accurady = history_dict[model_name][0].history['val_accuracy']
    val_loss = history_dict[model_name][0].history['val_loss']
    ax1.plot(val_accurady, label=model_name)
    ax2.plot(val_loss, label=model_name)
    
ax1.set_ylabel('validation accuracy')
ax2.set_ylabel('validation loss')
ax2.set_xlabel('epochs');

In [ ]:
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(10, 10))
plt.plot([0, 1], [0, 1], 'k--')

for model_name in history_dict:
    model = history_dict[model_name][1]
    
    y_pred = model.predict(x_test)
    fpr, tpr, threshold = roc_curve(y_test.ravel(), y_pred.ravel())
    
    plt.plot(fpr, tpr, label='{}, AUC = {:.3f}'.format(model_name, auc(fpr, tpr)))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate');